# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2019
topic: str = "FARMACIA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [x for x in range(14, 67, 1)] + [x for x in range(105, 131, 1)] + [x for x in range(167, 179, 1)] + [181, 182, 183]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. ¿Qué estructura química general tienen los
3,inhibidores de la catecol-O-metiltransferasa?:
4,
5,1. Propargilaminas.
6,"2. 3,5-Dihidroxiácidos."
7,3. Nitrocatecoles.
8,4. 5-Nitroimidazoles.
9,


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3,38,3.0,75,1.0,112,1,149,2.0
1,2,1,39,3.0,76,1.0,113,3,150,1.0
2,3,1,40,NaN,77,3.0,114,4,151,4.0
3,4,4,41,4.0,78,2.0,115,2,152,3.0
4,5,2,42,2.0,79,4.0,116,3,153,2.0
5,6,2,43,4.0,80,3.0,117,3,154,1.0
6,7,4,44,4.0,81,2.0,118,1,155,1.0
7,8,3,45,3.0,82,4.0,119,3,156,2.0
8,9,2,46,1.0,83,3.0,120,3,157,1.0
9,10,4,47,4.0,84,2.0,121,2,158,4.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [10]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [11]:
raw_exam_df.head(10)

0        1. ¿Qué estructura química general tienen los
1       inhibidores de la catecol-O-metiltransferasa?:
2                                  1. Propargilaminas.
3                              2. 3,5-Dihidroxiácidos.
4                                   3. Nitrocatecoles.
5                                4. 5-Nitroimidazoles.
6    2. La vildagliptina es un inhibidor de la dipe...
7       peptidasa IV (DPP IV) que se emplea en el tra-
8         tamiento de la diabetes tipo II. ¿Cuál es el
9        fragmento responsable de su unión covalente a
Name: text, dtype: object

In [12]:
raw_exam_df.tail(10)

1569                                                mano.
1570     3. Es un inhibidor de la síntesis de vitamina K.
1571     4. Es un inhibidor directo del factor X activado
1572                                                (Xa).
1573    185. ¿Qué fármaco se usa en cáncer de mama por su
1574                                   acción anti-HER2?:
1575                                         1. Imatinib.
1576                                        2. Lapatinib.
1577                                       3. Crizotinib.
1578                                      4. Vemurafenib.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [13]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [14]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. ¿Qué estructura química general tienen los ...
1                                  1. Propargilaminas.
2                              2. 3,5-Dihidroxiácidos.
3                                   3. Nitrocatecoles.
4                                4. 5-Nitroimidazoles.
5    2. La vildagliptina es un inhibidor de la dipe...
6                                 1. Cianopirrolidina.
7                               2. Difenilpropilamina.
8                                       3. Triptamina.
9                                     4. Sulfonilurea.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [15]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 2
Number of rows not ending with '.', ':' or '?' = 4
[12, 702, 791, 863]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [16]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [17]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [12, 791, ]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 12:
10    3. ¿Qué grupo es el responsable de la pérdida ...
11                 1. El grupo hidroxilo en posición 6.
12       2. Los grupos carbonilos de las posiciones 1 y
13                                                  11.
14                  3. El grupo amida de la posición 2.
Name: text, dtype: object
Id to fix 702:
700                               4. Conversión externa.
701    141. ¿En qué tipo de cromatografía líquida NO ...
702           1. Cromatografía de reparto en fase normal
703         2. Cromatografía de reparto en fase inversa.
704           3. Cromatografía de exclusión por tamaños.
Name: text, dtype: object
Id to fix 791:
789                            3. Cuantitativa discreta.
790                            4. Cuantitativa continua.
791    159. Señale cuáles de estos índices son coinci...
792                                                8, 9:
793                            1. La media y la mediana.
Name: text, dtype: object
Id to fix 863:
861    4. El 

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. ¿Qué estructura química general tienen los ...,1. Propargilaminas.,"2. 3,5-Dihidroxiácidos.",3. Nitrocatecoles.,4. 5-Nitroimidazoles.
1,2. La vildagliptina es un inhibidor de la dipe...,1. Cianopirrolidina.,2. Difenilpropilamina.,3. Triptamina.,4. Sulfonilurea.
2,3. ¿Qué grupo es el responsable de la pérdida ...,1. El grupo hidroxilo en posición 6.,2. Los grupos carbonilos de las posiciones 1 y11.,3. El grupo amida de la posición 2.,4. Los fragmentos de beta-hidroxicarbonilo.
3,4. En la bioactivación de la ciclofosfamida se...,1. Formaldehído.,2. Etanol.,3. Acetato sódico.,4. Acroleína.
4,5. El agente antimalárico cicloguanilo se admi...,1. Aumentar su solubilidad acuosa para su admi...,2. Favorecer el almacenamiento en tejidos gras...,3. Facilitar el metabolismo del fármaco por pa...,4. Disminuir su solubilidad acuosa para que ac...
5,6. ¿Qué parte de la estructura del indinavir e...,1. El anillo de piridina.,2. El fragmento de hidroxietileno.,3. La amina terciaria del anillo de piperidina.,4. El indano voluminoso sobre un enlace amida.
6,7. La loperamida es un antidiarreico que se di...,1. Antidepresivos tricíclicos.,2. Sartanes.,3. Benzodiazepinas.,4. Opioides.
7,8. ¿Cuál de las siguientes modificaciones indi...,1. Introducción de un grupo metilo en la posic...,2. Introducción de un átomo de flúor en la pos...,3. Eliminación del hidroxilo de la posición 21.,4. Introducción de un doble enlace adicional e...
8,9. Muchos fármacos y sus metabolitos de fase I...,1. Está catalizada por una transferasa.,2. La reacción tiene lugar con sustratos elect...,3. El ácido glucurónico interviene activado co...,4. La formación del glucurónido transcurre con...
9,"10. El atenolol, una ariloxipropanolamina, es ...",1. A la formación de una estructura rígida por...,"2. A la configuración del estereocentro, que e...",3. A la ausencia de los dos hidroxilos del cat...,4. Al sustituyente en la posición “para” del a...


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,1.0
2,3,1.0
3,4,4.0
4,5,2.0
5,6,2.0
6,7,4.0
7,8,3.0
8,9,2.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. ¿Qué estructura química general tienen los ...,1. Propargilaminas.,"2. 3,5-Dihidroxiácidos.",3. Nitrocatecoles.,4. 5-Nitroimidazoles.,3.0
1,2. La vildagliptina es un inhibidor de la dipe...,1. Cianopirrolidina.,2. Difenilpropilamina.,3. Triptamina.,4. Sulfonilurea.,1.0
2,3. ¿Qué grupo es el responsable de la pérdida ...,1. El grupo hidroxilo en posición 6.,2. Los grupos carbonilos de las posiciones 1 y11.,3. El grupo amida de la posición 2.,4. Los fragmentos de beta-hidroxicarbonilo.,1.0
3,4. En la bioactivación de la ciclofosfamida se...,1. Formaldehído.,2. Etanol.,3. Acetato sódico.,4. Acroleína.,4.0
4,5. El agente antimalárico cicloguanilo se admi...,1. Aumentar su solubilidad acuosa para su admi...,2. Favorecer el almacenamiento en tejidos gras...,3. Facilitar el metabolismo del fármaco por pa...,4. Disminuir su solubilidad acuosa para que ac...,2.0
5,6. ¿Qué parte de la estructura del indinavir e...,1. El anillo de piridina.,2. El fragmento de hidroxietileno.,3. La amina terciaria del anillo de piperidina.,4. El indano voluminoso sobre un enlace amida.,2.0
6,7. La loperamida es un antidiarreico que se di...,1. Antidepresivos tricíclicos.,2. Sartanes.,3. Benzodiazepinas.,4. Opioides.,4.0
7,8. ¿Cuál de las siguientes modificaciones indi...,1. Introducción de un grupo metilo en la posic...,2. Introducción de un átomo de flúor en la pos...,3. Eliminación del hidroxilo de la posición 21.,4. Introducción de un doble enlace adicional e...,3.0
8,9. Muchos fármacos y sus metabolitos de fase I...,1. Está catalizada por una transferasa.,2. La reacción tiene lugar con sustratos elect...,3. El ácido glucurónico interviene activado co...,4. La formación del glucurónido transcurre con...,2.0
9,"10. El atenolol, una ariloxipropanolamina, es ...",1. A la formación de una estructura rígida por...,"2. A la configuración del estereocentro, que e...",3. A la ausencia de los dos hidroxilos del cat...,4. Al sustituyente en la posición “para” del a...,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. ¿Qué estructura química general tienen los ...,1. Propargilaminas.,"2. 3,5-Dihidroxiácidos.",3. Nitrocatecoles.,4. 5-Nitroimidazoles.,3,2019
1,2. La vildagliptina es un inhibidor de la dipe...,1. Cianopirrolidina.,2. Difenilpropilamina.,3. Triptamina.,4. Sulfonilurea.,1,2019
2,3. ¿Qué grupo es el responsable de la pérdida ...,1. El grupo hidroxilo en posición 6.,2. Los grupos carbonilos de las posiciones 1 y11.,3. El grupo amida de la posición 2.,4. Los fragmentos de beta-hidroxicarbonilo.,1,2019
3,4. En la bioactivación de la ciclofosfamida se...,1. Formaldehído.,2. Etanol.,3. Acetato sódico.,4. Acroleína.,4,2019
4,5. El agente antimalárico cicloguanilo se admi...,1. Aumentar su solubilidad acuosa para su admi...,2. Favorecer el almacenamiento en tejidos gras...,3. Facilitar el metabolismo del fármaco por pa...,4. Disminuir su solubilidad acuosa para que ac...,2,2019
5,6. ¿Qué parte de la estructura del indinavir e...,1. El anillo de piridina.,2. El fragmento de hidroxietileno.,3. La amina terciaria del anillo de piperidina.,4. El indano voluminoso sobre un enlace amida.,2,2019
6,7. La loperamida es un antidiarreico que se di...,1. Antidepresivos tricíclicos.,2. Sartanes.,3. Benzodiazepinas.,4. Opioides.,4,2019
7,8. ¿Cuál de las siguientes modificaciones indi...,1. Introducción de un grupo metilo en la posic...,2. Introducción de un átomo de flúor en la pos...,3. Eliminación del hidroxilo de la posición 21.,4. Introducción de un doble enlace adicional e...,3,2019
8,9. Muchos fármacos y sus metabolitos de fase I...,1. Está catalizada por una transferasa.,2. La reacción tiene lugar con sustratos elect...,3. El ácido glucurónico interviene activado co...,4. La formación del glucurónido transcurre con...,2,2019
9,"10. El atenolol, una ariloxipropanolamina, es ...",1. A la formación de una estructura rígida por...,"2. A la configuración del estereocentro, que e...",3. A la ausencia de los dos hidroxilos del cat...,4. Al sustituyente en la posición “para” del a...,4,2019


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [34]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 94
Number of rows in filtered df = 94


**2. Showing the filtered DataFrame**

Using head and tail

In [35]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,14. ¿Cuál de las siguientes especies de Candid...,1. Candida albicans.,2. Candida auris.,3. Candida krusei.,4. Candida parapsilosis.,2,2019
1,15. ¿Qué especie de Aspergillus presenta resis...,1. A. fumigatus.,2. A. terreus.,3. A. niger.,4. A. versicolor.,2,2019
2,16. ¿Qué antifúngico utilizaría para el tratam...,1. Fluconazol.,2. Itraconazol.,3. Voriconazol.,4. Posaconazol.,4,2019
3,17. Señale la respuesta correcta sobre Bordete...,1. Es el agente causal de la difteria.,2. La infección que más comúnmente causa es la...,3. El tratamiento de elección de las infeccion...,"4. Produce distintos factores de virulencia, c...",4,2019
4,18. ¿Qué microorganismo es el causante de la f...,1. Rickettsia typhi.,2. Salmonella serotipo Typhi.,3. Rickettsia prowazeckii.,4. Shigella dysenteriae.,2,2019
5,19. El síndrome de Lady Windermere hace refere...,1. Mycobacterium avium complex.,2. Mycobacterium tuberculosis complex.,3. Mycobacterium xenopi.,4. Mycobacterium kansasii.,1,2019
6,20. ¿Cuál es el antibiótico de elección para e...,1. Ampicilina.,2. Ceftriaxona.,3. Cefepima.,4. Fosfomicina.,1,2019
7,21. ¿Cuál de las siguientes opciones es FALSA ...,1. Es parcialmente ácido-alcohol resistente.,2. En la tinción de Gram se observan bacilos g...,"3. Puede causar enfermedad pulmonar, cutánea o...",4. Las sulfamidas son antibióticos de elección...,2,2019
8,22. Señale la opción FALSA respecto a Legionella:,1. La infección se adquiere por inhalación de ...,2. La detección del antígeno en orina para el ...,3. Legionella pneumophila serogrupo 2 causa má...,4. El levofloxacino y la azitromicina son fárm...,3,2019
9,23. ¿Cuál de las siguientes relaciones entre m...,1. Mycobacterium ulcerans - Úlcera de Buruli.,2. Escherichia coli O157:H7 - Síndrome hemolít...,3. Yersinia pestis - Peste bubónica.,4. Clostridium botulinum - Gangrena gaseosa.,4,2019


In [36]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
84,172. Indique la respuesta INCORRECTA sobre el ...,1. Las regiones hipervariables del TCR sufren ...,2. Las cadenas que conforman el TCR pueden ser...,3. En el reordenamiento del TCR se produce el ...,4. El TCR-alfa-beta se genera por la recombina...,1,2019
85,173. ¿Cuál de las siguientes asociaciones entr...,1. Linfocitos T reguladores --- hongos --- lib...,2. Linfocitos TH2 --- alérgeno --- liberación ...,3. Linfocitos TH1 --- bacterias extracelulares...,4. Linfocitos TH17 --- bacterias intracelulare...,2,2019
86,174. Indique la afirmación INCORRECTA respecto...,1. CXCR5 dirige el movimiento de los linfocito...,2. CXCL13 determina la entrada de los linfocit...,3. CXCL8 es una quimiocina implicada en la qui...,4. CCR5 dirige la migración de los linfocitos ...,4,2019
87,175. ¿Qué reconoce un TCR?:,1. Un antígeno en su forma nativa.,2. Un péptido antigénico asociado a una molécu...,"3. Un antígeno de cualquier naturaleza, siempr...",4. Patrones moleculares sobre la superficie de...,2,2019
88,176. ¿Cuál de estas leyes siguen mejor la dist...,1. La ley hipogeométrica.,2. La ley de Poisson.,3. La ley de Norman.,4. La ley de Kruger.,2,2019
89,177. Las reacciones en el centro germinal:,1. Se producen en ausencia de linfocitos T.,2. Requieren el reclutamiento de neutrófilos y...,3. Son necesarias para la diferenciación a cél...,4. Se requieren para la maduración de la afini...,4,2019
90,178. ¿Cuál de los siguientes bacilos gramnegat...,1. Pseudomonas aeruginosa.,2. Acinetobacter baumanii.,3. Burkholderia cepacia.,4. Moraxella catarrhalis.,2,2019
91,181. Respecto a la diabetes mellitus indique l...,1. La diabetes mellitus tipo 2 está relacionad...,2. La diabetes mellitus tipo 1 cursa con hiper...,3. La diabetes mellitus tipo 1 es de aparición...,4. En la diabetes mellitus tipo 2 la cetoacido...,3,2019
92,182. La deficiencia de carbamoil-fosfato sinte...,1. El ciclo de la urea.,2. El ciclo de Cori.,3. El ciclo glucosa-alanina.,4. El ciclo de los ácidos tricarboxílicos.,1,2019
93,183. ¿Qué fármacos se utilizan tanto en el tra...,1. Lamivudina y tenofovir.,2. Lamivudina y darunavir.,3. Telbivudina y darunavir.,4. Tenofovir y darunavir.,1,2019


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [37]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [38]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [39]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [40]:

with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()